# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
city_data_df = pd.read_csv("..\WeatherPy\output_data\cities.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.12,-134.97,74.26,81.0,100.0,20.51,PF,1970-01-01 00:00:01.603657851
1,1,taoudenni,22.68,-3.98,88.47,12.0,0.0,7.29,ML,1970-01-01 00:00:01.603657797
2,2,hermanus,-34.42,19.23,61.00,79.0,86.0,1.01,ZA,1970-01-01 00:00:01.603657865
3,3,saskylakh,71.92,114.08,8.02,93.0,100.0,11.25,RU,1970-01-01 00:00:01.603657853
4,4,aracaju,-10.91,-37.07,82.40,74.0,40.0,14.99,BR,1970-01-01 00:00:01.603657903
...,...,...,...,...,...,...,...,...,...,...
598,598,karpathos,35.51,27.21,71.60,78.0,99.0,9.17,GR,1970-01-01 00:00:01.603658057
599,599,port macquarie,-31.43,152.92,63.00,89.0,95.0,1.99,AU,1970-01-01 00:00:01.603657855
600,600,ranong,9.97,98.63,76.87,87.0,92.0,6.04,TH,1970-01-01 00:00:01.603658058
601,601,gagarin,55.55,35.00,45.66,82.0,96.0,6.08,RU,1970-01-01 00:00:01.603658058


In [9]:
cleaned_city_data_df = city_data_df.dropna()
del cleaned_city_data_df["Unnamed: 0"]

In [10]:
cleaned_city_data_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity      float64
Cloudiness    float64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [11]:
# Convert "Date" column to "datetime64" datatype
cleaned_city_data_df['Date'] = pd.to_datetime(cleaned_city_data_df['Date'])

C:\Users\reiki\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
cleaned_city_data_df.dtypes

City                  object
Lat                  float64
Lng                  float64
Max Temp             float64
Humidity             float64
Cloudiness           float64
Wind Speed           float64
Country               object
Date          datetime64[ns]
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = cleaned_city_data_df[["Lat", "Lng"]]

# Humidity
humidity = cleaned_city_data_df["Humidity"]

In [14]:
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=300,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

#   A max temperature lower than 80 degrees but higher than 70.

#   Wind speed less than 10 mph.

#   Zero cloudiness.

#   Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
narrowed_df = cleaned_city_data_df.loc[(cleaned_city_data_df["Max Temp"] > 70) &
                                       (cleaned_city_data_df["Max Temp"] < 80) &
                                       (cleaned_city_data_df["Wind Speed"] < 10) &
                                       (cleaned_city_data_df["Cloudiness"] == 0)]
narrowed_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
100,chlorakas,34.80,32.41,71.60,46.0,0.0,9.17,CY,1970-01-01 00:00:01.603657926
104,keti bandar,24.14,67.45,77.81,55.0,0.0,6.38,PK,1970-01-01 00:00:01.603657927
114,noumea,-22.28,166.46,78.01,69.0,0.0,2.24,NC,1970-01-01 00:00:01.603657820
161,ikwiriri,-7.96,38.97,73.90,87.0,0.0,2.66,TZ,1970-01-01 00:00:01.603657940
374,olpad,21.33,72.75,77.92,49.0,0.0,6.04,IN,1970-01-01 00:00:01.603657996
435,salalah,17.02,54.09,77.00,57.0,0.0,4.70,OM,1970-01-01 00:00:01.603657832
469,ambilobe,-13.20,49.05,73.08,65.0,0.0,7.20,MG,1970-01-01 00:00:01.603657755
470,beira,-19.84,34.84,75.20,88.0,0.0,5.82,MZ,1970-01-01 00:00:01.603658023
582,bilma,18.69,12.92,72.52,21.0,0.0,3.36,NE,1970-01-01 00:00:01.603657872


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = narrowed_df

hotel_df["Hotel Name"] = ""

hotel_df

C:\Users\reiki\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
100,chlorakas,34.80,32.41,71.60,46.0,0.0,9.17,CY,1970-01-01 00:00:01.603657926,
104,keti bandar,24.14,67.45,77.81,55.0,0.0,6.38,PK,1970-01-01 00:00:01.603657927,
114,noumea,-22.28,166.46,78.01,69.0,0.0,2.24,NC,1970-01-01 00:00:01.603657820,
161,ikwiriri,-7.96,38.97,73.90,87.0,0.0,2.66,TZ,1970-01-01 00:00:01.603657940,
374,olpad,21.33,72.75,77.92,49.0,0.0,6.04,IN,1970-01-01 00:00:01.603657996,
435,salalah,17.02,54.09,77.00,57.0,0.0,4.70,OM,1970-01-01 00:00:01.603657832,
469,ambilobe,-13.20,49.05,73.08,65.0,0.0,7.20,MG,1970-01-01 00:00:01.603657755,
470,beira,-19.84,34.84,75.20,88.0,0.0,5.82,MZ,1970-01-01 00:00:01.603658023,
582,bilma,18.69,12.92,72.52,21.0,0.0,3.36,NE,1970-01-01 00:00:01.603657872,


In [17]:
# Loop through the hotel dataframe and find nearest hotel
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "hotel",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_data = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = "Nearest Hotel Not Found"

hotel_df

C:\Users\reiki\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
100,chlorakas,34.80,32.41,71.60,46.0,0.0,9.17,CY,1970-01-01 00:00:01.603657926,Paphos
104,keti bandar,24.14,67.45,77.81,55.0,0.0,6.38,PK,1970-01-01 00:00:01.603657927,Keti Bandar
114,noumea,-22.28,166.46,78.01,69.0,0.0,2.24,NC,1970-01-01 00:00:01.603657820,Noumea
161,ikwiriri,-7.96,38.97,73.90,87.0,0.0,2.66,TZ,1970-01-01 00:00:01.603657940,Ikwiriri
374,olpad,21.33,72.75,77.92,49.0,0.0,6.04,IN,1970-01-01 00:00:01.603657996,Olpad
435,salalah,17.02,54.09,77.00,57.0,0.0,4.70,OM,1970-01-01 00:00:01.603657832,Salalah
469,ambilobe,-13.20,49.05,73.08,65.0,0.0,7.20,MG,1970-01-01 00:00:01.603657755,Ambilobe
470,beira,-19.84,34.84,75.20,88.0,0.0,5.82,MZ,1970-01-01 00:00:01.603658023,Beira
582,bilma,18.69,12.92,72.52,21.0,0.0,3.36,NE,1970-01-01 00:00:01.603657872,Bilma


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, 
                             info_box_content= hotel_info)

# Add layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))